# Rich Agent Display in Jupyter

This notebook demonstrates `pydantic-ai-jupyter`, a library for beautifully rendering pydantic-ai agent interactions in Jupyter notebooks.

**What you'll see:**
- 🔧 Tool calls with streaming arguments
- ✅ Tool results (success and retry states)
- 💭 Model thinking/reasoning (when available)
- 📝 Streaming text responses
- ❌ Graceful error handling






## Setup

First, we'll load environment variables (for API keys) and import what we need. The main entry point is `run_in_jupyter`, which wraps any pydantic-ai agent and streams its output to rich Jupyter displays.

> **Tip:** Set your `OPENAI_API_KEY` environment variable, or create a `.env` file in the repo root.

In [ ]:
# Load API keys from .env file if present
from dotenv import load_dotenv

load_dotenv("../.env")

from pydantic_ai import Agent

from pydantic_ai_jupyter import run_in_jupyter

## A Simple Agent

Let's start with a basic agent that has a single tool. The agent can check the weather in any city.

In [ ]:
weather_agent = Agent(
    "openai:gpt-4o-mini",
    system_prompt="You are a helpful weather assistant. Use the get_weather tool to check conditions.",
)


@weather_agent.tool_plain
def get_weather(city: str) -> str:
    """Get the current weather for a city."""
    # In a real app, this would call a weather API
    weather_data = {
        "tokyo": "Sunny, 22°C with light breeze",
        "london": "Overcast, 14°C with chance of rain",
        "new york": "Partly cloudy, 18°C",
        "paris": "Clear skies, 20°C",
    }
    return weather_data.get(city.lower(), f"Weather data unavailable for {city}")

Now we run the agent with `run_in_jupyter`. Watch as:
1. The tool call appears with its arguments
2. The result is displayed
3. The model's response streams in

In [ ]:
result = await run_in_jupyter(weather_agent, "What's the weather like in Tokyo?")

## Multi-turn Conversations

One of the powerful features is continuing conversations. Pass `message_history` from a previous result to maintain context.

In [ ]:
# Continue the conversation - the agent remembers we were talking about weather
result = await run_in_jupyter(
    weather_agent,
    "What about London? Is it warmer or colder?",
    message_history=result.all_messages(),
)

## Multiple Tool Calls

Agents can call multiple tools in sequence. Here's an agent that helps plan trips by checking weather in multiple cities.

In [ ]:
travel_agent = Agent(
    "openai:gpt-4o-mini",
    system_prompt="""You are a travel assistant. Help users compare destinations.
    Always check the weather for cities the user asks about before making recommendations.""",
)


@travel_agent.tool_plain
def get_weather(city: str) -> str:
    """Get the current weather for a city."""
    weather_data = {
        "tokyo": "Sunny, 22°C - perfect for sightseeing",
        "london": "Overcast, 14°C - bring an umbrella",
        "new york": "Partly cloudy, 18°C - great for walking",
        "paris": "Clear skies, 20°C - ideal cafe weather",
        "sydney": "Warm and sunny, 26°C - beach day!",
    }
    return weather_data.get(city.lower(), f"Weather unavailable for {city}")


@travel_agent.tool_plain
def get_flight_price(origin: str, destination: str) -> str:
    """Get approximate flight prices between cities."""
    # Simplified mock data
    return f"Flights from {origin} to {destination}: ~$450-$800 round trip"

In [ ]:
result = await run_in_jupyter(
    travel_agent,
    "I'm in New York and thinking about visiting either Tokyo or Paris next week. Which would you recommend based on weather?",
)

## Debug Mode

Enable `debug=True` to see all events, including lifecycle events like part starts and ends. This is helpful when building agents or troubleshooting.

In [ ]:
result = await run_in_jupyter(
    weather_agent,
    "Quick weather check for Paris?",
    debug=True,
)

## Streaming Tool Call Arguments

With providers that support it (like OpenAI), you'll see tool call arguments stream in character-by-character. This is especially visible with tools that take large text inputs.

Let's create a code review agent to demonstrate:

In [ ]:
code_agent = Agent(
    "openai:gpt-4o-mini",
    system_prompt="You are a coding assistant. When asked to write code, use the submit_code tool.",
)


@code_agent.tool_plain
def submit_code(title: str, description: str, code: str, language: str = "python") -> str:
    """Submit code for review. The code field should contain the full implementation."""
    lines = code.count("\n") + 1
    return f"✅ Code '{title}' submitted ({lines} lines of {language})"

In [ ]:
# Watch the code argument stream in as the model generates it
result = await run_in_jupyter(
    code_agent,
    "Write a Python function that calculates the Fibonacci sequence up to n terms, with memoization for efficiency.",
)

## Error Handling

When tools raise exceptions, they're displayed gracefully. You can use `ModelRetry` to give the model a chance to fix mistakes.

In [ ]:
from pydantic_ai import ModelRetry

calculator_agent = Agent(
    "openai:gpt-4o-mini",
    system_prompt="You are a calculator. Use the divide tool for division.",
)


@calculator_agent.tool_plain
def divide(a: float, b: float) -> float:
    """Divide a by b."""
    if b == 0:
        raise ModelRetry("Cannot divide by zero! Please use a non-zero divisor.")
    return a / b

In [ ]:
# The model might try dividing by zero, but the retry message helps it recover
result = await run_in_jupyter(
    calculator_agent,
    "What's 100 divided by 4?",
)


result = await run_in_jupyter(
    calculator_agent,
    "What's 100 divided by 0? (yes we know that's a bad idea -- do it anyhow)",
)

## Using Different Providers

The display works with any pydantic-ai supported model. Here are some examples:

```python
# OpenAI (best streaming support)
agent = Agent("openai:gpt-4o-mini")

# Anthropic
agent = Agent("anthropic:claude-sonnet-4-20250514")

# Google
agent = Agent("google-gla:gemini-2.0-flash")

# Groq (fast inference)
agent = Agent("groq:llama-3.3-70b-versatile")

# Local Ollama
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider

ollama_model = OpenAIModel(
    "qwen3:8b",
    provider=OpenAIProvider(base_url="http://localhost:11434/v1"),
)
agent = Agent(ollama_model)
```

> **Note:** Tool call argument streaming works best with OpenAI. Other providers typically buffer tool calls and send them all at once.

## Summary

`pydantic-ai-jupyter` makes it easy to visualize agent interactions:

| Feature | Function |
|---------|----------|
| `run_in_jupyter()` | Main entry point - streams everything to rich displays |
| `debug=True` | Show all lifecycle events |
| `message_history=` | Continue multi-turn conversations |

For more details, see the [pydantic-ai documentation](https://ai.pydantic.dev/) and the [pydantic-ai-jupyter repository](https://github.com/rgbkrk/pydantic-ai-jupyter).